In [1]:
import os

In [2]:
os.chdir("E:/dibimbing/Portfolio/Bank-Loan-Default-Prediction")

In [3]:
%pwd

'E:\\dibimbing\\Portfolio\\Bank-Loan-Default-Prediction'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/JuliusHmto/Bank-Loan-Default-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="JuliusHmto"
os.environ["MLFLOW_TRACKING_PASSWORD"]="64f6b16c2f95339023b7e5ba35bbdc160d13e30d"

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    x_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
from src.loanDefault.constants import *
from src.loanDefault.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            x_test_data_path=config.x_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.Default,
            mlflow_uri="https://dagshub.com/JuliusHmto/Bank-Loan-Default-Prediction.mlflow",
           
        )

        return model_evaluation_config


In [8]:
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        # Compute confusion matrix and classification report
        conf_matrix = confusion_matrix(actual, pred)
        class_report = classification_report(actual, pred, output_dict=True)

        return conf_matrix, class_report

    def log_into_mlflow(self):
        test_x = pd.read_csv(self.config.x_test_data_path)
        test_y = pd.read_csv(self.config.y_test_data_path)

        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_classes = model.predict(test_x)

            # Evaluate metrics for classification
            conf_matrix, class_report = self.eval_metrics(test_y, predicted_classes)

            # Save classification report as JSON using save_json
            metrics_file_path = Path(self.config.metric_file_name)
            save_json(path=metrics_file_path, data=class_report)

            # Log the classification report metrics into mlflow
            for label, metrics in class_report.items():
                if isinstance(metrics, dict):  # Skip the 'accuracy' entry which is not a dict
                    mlflow.log_metric(f'{label}_precision', metrics['precision'])
                    mlflow.log_metric(f'{label}_recall', metrics['recall'])
                    mlflow.log_metric(f'{label}_f1-score', metrics['f1-score'])
                    mlflow.log_metric(f'{label}_support', metrics['support'])

            # Log the overall accuracy
            mlflow.log_metric('accuracy', class_report['accuracy'])

            mlflow.log_params(self.config.all_params)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-08 14:40:41,684: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-08 14:40:41,687: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-08 14:40:41,691: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-08 14:40:41,693: INFO: common: created directory at: artifacts]
[2024-10-08 14:40:41,694: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-08 14:40:48,407: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/10/08 14:40:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBModel' already exists. Creating a new version of this model...
2024/10/08 14:41:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBModel, version 2
Created version '2' of model 'XGBModel'.
2024/10/08 14:41:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-lamb-104 at: https://dagshub.com/JuliusHmto/Bank-Loan-Default-Prediction.mlflow/#/experiments/0/runs/b9ca3ba88160413eb4cedd3b0f00ada1.
2024/10/08 14:41:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuliusHmto/Bank-Loan-Default-Prediction.mlflow/#/experiments/0.
